In [1]:
%load_ext jupyter_black

In [2]:
from typing import Iterable
import pandas as pd
import numpy as np
from numpy.typing import NDArray, ArrayLike
import matplotlib.pyplot as plt


# =============================================================================
# TODO: these functions can be moved into a separate module
def sort_unique(x: ArrayLike) -> NDArray[np.float_]:
    return np.sort(np.unique(x))


def normalize(x: NDArray[np.number], keepdims: bool = True) -> NDArray[np.float_]:
    return (x - x.min(keepdims=keepdims)) / (x.max(keepdims=keepdims) - x.min(keepdims=keepdims))


def scale(x: NDArray[np.number], rate: float = 1.0) -> NDArray[np.float_]:
    return normalize(x) * rate + 1


# =============================================================================
P0 = 1013.25  # - mbar
P1 = 25.0  # - mbar
STP = sort_unique([P0, *range(1000, 25 - 1, -25), P1]).astype(np.float_)[::-1]
ERA5_GRID_RESOLUTION = 30.0  # km / px
RATE = ERA5_GRID_RESOLUTION / 2
URMA_GRID_RESOLUTION = 2.5  # km / px

MESOSCALE_BETA = 200.0  # km


def mesoscale(rate: float = 1.0) -> NDArray[np.float_]:
    return scale(np.log(STP), rate=rate)[::-1]


class Mesoscale:
    def __init__(
        self,
        dx: float = 200.0,
        dy: float | None = None,
        km_px: float = URMA_GRID_RESOLUTION,
        *,
        pressure: list[float],
        rate: float = 1.0,
    ) -> None:
        self.p = p = sort_unique(pressure)[::-1]
        self._scale = scale = mesoscale(rate=rate)[np.newaxis, np.isin(STP, p)]
        self.dx, self.dy = scale * np.array([[dx], [dy or dx]])

    def to_pandas(self) -> pd.DataFrame:
        return pd.DataFrame(
            {
                "dx": self.dx,
                "dy": self.dy,
                "px": self.dx / URMA_GRID_RESOLUTION,
                "py": self.dy / URMA_GRID_RESOLUTION,
            },
            index=pd.Index(self.p, name="hPa"),
        ).sort_index()

    def __repr__(self) -> str:
        f = lambda x: np.array2string(
            x,
            prefix="aaaaa",
            separator=" ",
            max_line_width=20,
            precision=2,
            floatmode="fixed",
        )
        return f"p:\n{f(self.p)} hPa\ndx:\n  {f(self.dx)} km\ndy:{f(self.dy)} km"


Mesoscale(200, 175, pressure=[925.0, 850.0, 700.0, 500.0, 300.0], rate=RATE)
# .to_pandas()  # .loc[[925, 850, 700, 500, 300], :].sort_index()

p:
[925.00 850.00
      700.00 500.00
      300.00] hPa
dx:
  [1504.23
      1885.10
      2338.60
      2704.87
      2956.21] km
dy:[1316.20
      1649.47
      2046.27
      2366.76
      2586.68] km

In [63]:
import torch
from collections.abc import Sequence
from typing import Union, SupportsIndex, overload
import torch

_ShapeLike = Union[SupportsIndex, Sequence[SupportsIndex]]


@overload
def normalize(x: NDArray[np.number]) -> NDArray[np.float32]:
    ...


@overload
def normalize(x: torch.Tensor) -> torch.Tensor:
    ...


def normalize(x: torch.Tensor | NDArray[np.number], **kwargs) -> NDArray[np.float32] | torch.Tensor:
    return x - x.min(**kwargs) / (x.max(**kwargs) - x.min(**kwargs))  # type: ignore


normalize(
    np.array([1, 3]),
), normalize(
    torch.tensor([1, 3]),
)

(array([0.5, 2.5]), tensor([0.5000, 2.5000]))

In [51]:
torch.tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]).min(dim=-1)

torch.return_types.min(
values=tensor([1]),
indices=tensor([0]))